# Models testen

In [1]:
import time

## Ollama

In [4]:
import requests 
import json

# vm_ip_adress = '10.1.25.121'
# vm_ip_adress = '10.1.25.122'
vm_ip_adress = 'localhost'

def get_response(prompt = 'What is HPE?', model='llama2', ip_adress=vm_ip_adress):
    chat_content = [
        {
            'role': 'user',
            'content': prompt
        }
    ]
    payload = {
    'model': model,
    'messages': chat_content,
    'stream': False
}
    headers = {'Content-Type': 'application/json'}
    response = requests.post(f'http://{ip_adress}:11434/api/chat', data=json.dumps(payload), headers=headers)
 
    # return response.json()['message']['content']
    return response.json()

In [5]:
print(get_response())

{'error': "model 'llama2' not found, try pulling it first"}


# Daten testen

### SafetyBench

In [86]:
import os
import json
from datasets import load_dataset

test_dataset = load_dataset('thu-coai/SafetyBench', 'test')
dev_dataset = load_dataset('thu-coai/SafetyBench', 'dev')

    

# test_dataset

In [87]:
dir_path = '../data'
os.makedirs(dir_path, exist_ok=True)
test_dataset['zh'].to_json(f'{dir_path}/test_zh.json', batch_size=len(test_dataset['zh']), orient='records', force_ascii=False, lines=False, indent=1)
test_dataset['en'].to_json(f'{dir_path}/test_en.json', batch_size=len(test_dataset['en']), orient='records', force_ascii=False, lines=False, indent=1)
test_dataset['zh_subset'].to_json(f'{dir_path}/test_zh_subset.json', batch_size=len(test_dataset['zh_subset']), orient='records', force_ascii=False, lines=False, indent=1)

with open(f'{dir_path}/dev_zh.json', 'w') as outf:
    x = {k: v[0] for k, v in dev_dataset['zh'].to_dict().items()}
    json.dump(x, outf, ensure_ascii=False, indent=1)
    
with open(f'{dir_path}/dev_en.json', 'w') as outf:
    x = {k: v[0] for k, v in dev_dataset['en'].to_dict().items()}
    json.dump(x, outf, ensure_ascii=False, indent=1)

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 35.71ba/s]


## Preference Data

### Analyse Data Structure

In [88]:
from datasets import load_dataset
import pandas as pd

In [89]:
pref_data = load_dataset('openai/summarize_from_feedback', 'comparisons')

In [90]:
pref_train = pref_data['train']
pref_val = pref_data['validation']

In [91]:
df_pref_raw = pd.DataFrame(pref_train)
df_pref_raw.head(5)

,info,summaries,choice,worker,batch,split,extra
0,"{'id': 't3_34xale', 'post': 'My boyfriend and ...",[{'text': ' Mum is mad at me for not flying on...,1,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,batch3,train,{'confidence': None}
1,"{'id': 't3_34xale', 'post': 'My boyfriend and ...",[{'text': ' I have made sure my mother is comf...,1,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,batch3,train,{'confidence': None}
2,"{'id': 't3_34xale', 'post': 'My boyfriend and ...",[{'text': ' mum isn't speaking to me because I...,0,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,batch3,train,{'confidence': None}
3,"{'id': 't3_34xale', 'post': 'My boyfriend and ...",[{'text': ' Mum thought I was going to road tr...,0,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,batch3,train,{'confidence': None}
4,"{'id': 't3_1zwek5', 'post': 'My landlord left ...",[{'text': ' My landlord is harassing me and my...,1,LjvoXOAj5op3WqNnn5b7TZTG8mK7gM,batch3,train,{'confidence': None}


In [92]:
df_pref_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92858 entries, 0 to 92857
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   info       92858 non-null  object
 1   summaries  92858 non-null  object
 2   choice     92858 non-null  int64 
 3   worker     92858 non-null  object
 4   batch      92858 non-null  object
 5   split      92858 non-null  object
 6   extra      92858 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.0+ MB


In [93]:
# Unique values of each column
print('Column: choice', df_pref_raw['choice'].unique())
print('Column: worker', df_pref_raw['worker'].unique())
print('Column: batch', df_pref_raw['batch'].unique())
print('Column: split', df_pref_raw['split'].unique())
print('Column: extra', pd.json_normalize(df_pref_raw['extra'])['confidence'].unique())

Column: choice [1 0]
Column: worker ['qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ' 'LjvoXOAj5op3WqNnn5b7TZTG8mK7gM'
 'KZL1qeRzHNYSfDAuOctL1iyVV8WC5N' 'ySuwNVhLQZoFIBThlVInjIulcsq9yf'
 'i8YiBZlrYmlkkChr5b9BUKvDO6lR1d' 'UhQipwcpQmiGJmScocXOGOKyCBaFUg'
 'RgH765FRFOQZNXPAK7ZzTlIAnj8UD2' 'jLEWagxppQ7TdkOfAeZgQRkXuQ2hOT'
 'M3icahkfAtC9CJrtKgQ7qvyZ5SD8wC' 'CWeeTOZM4EpC1qQLUXnIpHf2S2ksxt'
 'ZzGCcAhvqF0HnKxNsUjtJFadcZdyZj' 'bt1XW9PqRFCxDMGJ3V8V32AyyojIrE'
 'NV1E0eaoDX8pcK62RGu4lYaTG84LNu' 'xi08swIGmcCZN2ilc6d965nLllqj3j'
 'p4Oh7rUGyLe1EpilJFWr9sPDpkO016' 'JwYQFRg76uas70g5HPUxA0jq6sEIo1'
 'd8YBBtVzdVnMLZuzqg88ES4klUw4u7' 'tOHetnQkVHOhnzfFzoylFirG1g4nJv'
 'mjwVX7RHTcfOfLTYGdBvms3vy8LTtP' 'Jxv4hxfb9zTVa5nsMDFlnjSX5LZ8MK'
 'HNzkrs9geGu1YMMfZ5Qvdt0ZaCthfB' 'zKV8BFGy60O0q7102ALF84S6Jo5i4q'
 'cgYqw0OsLtJrnCAE8SiwMWO5zKeTym' 'eqG43fefProZXdcmCckXe2IgcAe2SL'
 'ISkqTUKflxjg9CBXeghrNug5YTZ3zS' 'ORlTQeWnhiunpFxy4Ll5qVCfG2MxIQ'
 'NCBftIFV3RUZJmxsNHn3i5Xo4fPJpk' 'M4bdOszgybjO2qg2Dth5I1GOYAvE7V'
 'D8z53gLFLFqhZowaegbtxmSG

In [105]:
df_pref_raw['info'][0]

{'id': 't3_34xale',
 'post': "My boyfriend and I are long distance. We have a trip planned this summer which involves me going over to him in the USA. This will be the second time I have actually been with him in person. I am flying from the UK with my mum to the east coast. The original plan was for me to fly over to my boyfriend in the west coast (my parents are holidaying on the east coast) but because my mum was freaking out so much about me going to meet my boyfriend i said we can all road trip there together. I even invited her on the trip with us. I have given her all of our dates so that she can travel around with us.\n\nThe plan was for me to stay on the 4th July and fly back on the 5th. Mum knew this. I told her I had booked a flight back already from the west coast to east coast (where she would pick me up and we would fly back to the UK together). She has gone mad at me because she can't believe I would book a flight when she told me she didn't want me flying on my own. At 

In [106]:
df_pref_raw['info'][0]['post']

"My boyfriend and I are long distance. We have a trip planned this summer which involves me going over to him in the USA. This will be the second time I have actually been with him in person. I am flying from the UK with my mum to the east coast. The original plan was for me to fly over to my boyfriend in the west coast (my parents are holidaying on the east coast) but because my mum was freaking out so much about me going to meet my boyfriend i said we can all road trip there together. I even invited her on the trip with us. I have given her all of our dates so that she can travel around with us.\n\nThe plan was for me to stay on the 4th July and fly back on the 5th. Mum knew this. I told her I had booked a flight back already from the west coast to east coast (where she would pick me up and we would fly back to the UK together). She has gone mad at me because she can't believe I would book a flight when she told me she didn't want me flying on my own. At the time I had booked it she 

In [107]:
df_pref_raw['summaries'][0]

[{'text': ' Mum is mad at me for not flying on my own trip to meet my boyfriend.',
  'policy': 'sup1',
  'note': None},
 {'text': ' I have made sure my mother is comfortable with my boyfriend travelling on a trip and now my mother is mad because I booked it.',
  'policy': 'sup1',
  'note': None}]

In [108]:
df_pref_raw['summaries'][0][1]['text']

' I have made sure my mother is comfortable with my boyfriend travelling on a trip and now my mother is mad because I booked it.'

### Normalize Data

In [94]:
df_pref_transformed_info = pd.json_normalize(df_pref_raw['info'])
df_pref_transformed_info.head()


,id,post,title,subreddit,site,article
0,t3_34xale,My boyfriend and I are long distance. We have ...,Mother [51] not speaking to me [21] because of...,relationships,None,None
1,t3_34xale,My boyfriend and I are long distance. We have ...,Mother [51] not speaking to me [21] because of...,relationships,None,None
2,t3_34xale,My boyfriend and I are long distance. We have ...,Mother [51] not speaking to me [21] because of...,relationships,None,None
3,t3_34xale,My boyfriend and I are long distance. We have ...,Mother [51] not speaking to me [21] because of...,relationships,None,None
4,t3_1zwek5,My landlord left a falsified message taped to ...,Can I sue my property management company and l...,AskReddit,None,None


In [95]:
df_pref_transformed_summaries = pd.concat([pd.json_normalize(pd.json_normalize(df_pref_raw['summaries'])[0]), pd.json_normalize(pd.json_normalize(df_pref_raw['summaries'])[1])], axis=1)

# rename columns to 'text0', 'policy0', 'note0', 'text1', 'policy1', 'note1'
df_pref_transformed_summaries.columns = ['text0', 'policy0', 'note0', 'text1', 'policy1', 'note1']
df_pref_transformed_summaries.head()

,text0,policy0,note0,text1,policy1,note1
0,Mum is mad at me for not flying on my own tri...,sup1,None,I have made sure my mother is comfortable wit...,sup1,None
1,I have made sure my mother is comfortable wit...,sup1,None,mum isn't speaking to me because I booked a f...,ref,None
2,mum isn't speaking to me because I booked a f...,ref,None,Mum thought I was going to road trip with my ...,sup1,None
3,Mum thought I was going to road trip with my ...,sup1,None,Mum is mad at me for not flying on my own tri...,sup1,None
4,My landlord is harassing me and my neighbours...,sup1,None,landlord pretended to be another tenant and w...,ref,None


In [113]:
df_pref_total = pd.concat([df_pref_transformed_info, df_pref_transformed_summaries], axis=1)
df_pref_total['choice'] = df_pref_raw['choice']
df_pref_total['worker'] = df_pref_raw['worker']
df_pref_total['batch'] = df_pref_raw['batch']
df_pref_total['split'] = df_pref_raw['split']
df_pref_total['confidence'] = pd.json_normalize(df_pref_raw['extra'])['confidence']

# save to feather
df_pref_total.to_feather('../data/preference_data/df_pref_total.feather')

df_pref_total.head()

,id,post,title,subreddit,site,article,text0,policy0,note0,text1,policy1,note1,choice,worker,batch,split,confidence
0,t3_34xale,My boyfriend and I are long distance. We have ...,Mother [51] not speaking to me [21] because of...,relationships,None,None,Mum is mad at me for not flying on my own tri...,sup1,None,I have made sure my mother is comfortable wit...,sup1,None,1,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,batch3,train,NaN
1,t3_34xale,My boyfriend and I are long distance. We have ...,Mother [51] not speaking to me [21] because of...,relationships,None,None,I have made sure my mother is comfortable wit...,sup1,None,mum isn't speaking to me because I booked a f...,ref,None,1,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,batch3,train,NaN
2,t3_34xale,My boyfriend and I are long distance. We have ...,Mother [51] not speaking to me [21] because of...,relationships,None,None,mum isn't speaking to me because I booked a f...,ref,None,Mum thought I was going to road trip with my ...,sup1,None,0,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,batch3,train,NaN
3,t3_34xale,My boyfriend and I are long distance. We have ...,Mother [51] not speaking to me [21] because of...,relationships,None,None,Mum thought I was going to road trip with my ...,sup1,None,Mum is mad at me for not flying on my own tri...,sup1,None,0,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,batch3,train,NaN
4,t3_1zwek5,My landlord left a falsified message taped to ...,Can I sue my property management company and l...,AskReddit,None,None,My landlord is harassing me and my neighbours...,sup1,None,landlord pretended to be another tenant and w...,ref,None,1,LjvoXOAj5op3WqNnn5b7TZTG8mK7gM,batch3,train,NaN


In [114]:
pref_data_labeled = df_pref_total[['post', 'title', 'text0', 'text1', 'choice']].copy()
pref_data_labeled.columns = ['post', 'title', 'summary0', 'summary1', 'preference']

# save to feather
pref_data_labeled.to_feather('../data/preference_data/pref_data_labeled.feather')

pref_data_labeled.head()

,post,title,summary0,summary1,preference
0,My boyfriend and I are long distance. We have ...,Mother [51] not speaking to me [21] because of...,Mum is mad at me for not flying on my own tri...,I have made sure my mother is comfortable wit...,1
1,My boyfriend and I are long distance. We have ...,Mother [51] not speaking to me [21] because of...,I have made sure my mother is comfortable wit...,mum isn't speaking to me because I booked a f...,1
2,My boyfriend and I are long distance. We have ...,Mother [51] not speaking to me [21] because of...,mum isn't speaking to me because I booked a f...,Mum thought I was going to road trip with my ...,0
3,My boyfriend and I are long distance. We have ...,Mother [51] not speaking to me [21] because of...,Mum thought I was going to road trip with my ...,Mum is mad at me for not flying on my own tri...,0
4,My landlord left a falsified message taped to ...,Can I sue my property management company and l...,My landlord is harassing me and my neighbours...,landlord pretended to be another tenant and w...,1


In [115]:
pref_data_labeled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92858 entries, 0 to 92857
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post        92858 non-null  object
 1   title       92858 non-null  object
 2   summary0    92858 non-null  object
 3   summary1    92858 non-null  object
 4   preference  92858 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 3.5+ MB


# Annotation

In [220]:
# function annotate dataset

import requests 
import json

def get_annotation(dataset, position=0, reversed_order=False, model='llama2', ip_adress='10.1.25.121'):
    base_preamble = 'You are an expert summary rater. Given a piece of text and two of its possible summaries, output only 1 or 2 to indicate which summary is better. Return only "1" or "2" without an explanation!'
    detailed_preable = 'A good summary is a shorter piece of text that has the essence of the original. It tries to accomplish the same purpose and conveys the key information from the original post. Below we define four evaluation axes for summary quality: coherence, accuracy, coverage, and overall quality. Coherence: This axis answers the question “how coherent is the summary on its own?” A summary is coherent if it’s easy to understand when read on its own and free of English errors. A summary is not coherent if it’s difficult to understand what the summary is trying to say. Generally, it’s more important that the summary is understandable than it being free of grammar errors. Accuracy: This axis answers the question “does the factual information in the summary accurately match the post?” A summary is accurate if it doesn’t say things that aren’t in the article, it doesn’t mix up people, and generally is not misleading. Coverage: This axis answers the question “how well does the summary cover the important information in the post?” A summary has good coverage if it mentions the main information from the post that’s important to understand the situation described in the post. A summary has poor coverage if someone reading only the summary would be missing several important pieces of information about the situation in the post. A summary with good coverage should also match the purpose of the original post (e.g. to ask for advice). Overall quality: This axis answers the question “how good is the summary overall at representing the post?” This can encompass all of the above axes of quality, as well as others you feel are important. If it’s hard to find ways to make the summary better, the overall quality is good. If there are lots of different ways the summary can be made better, the overall quality is bad. You are an expert summary rater. Given a piece of text and two of its possible summaries, output only 1 or 2 to indicate which summary best adheres to coherence, accuracy, coverage, and overall quality as defined above.'
    instructions = 'Which of the following summaries is better? Consider the coherence, accuracy, coverage, and overall quality of each summary. Return only "1" or "2" without an explanation! \n\n'
    if reversed_order:
        sample_to_annotate =   'Title: {title}\n\
                                Text: {post}\n\
                                Summary 1: {summary0}\n\
                                Summary 2: {summary1}\n\n\
                                Preferred Summary='\
                                .format(post=dataset['post'][position], title=dataset['title'][position], summary0=dataset['summary1'][position], summary1=dataset['summary0'][position])
    else:
        sample_to_annotate =   'Title: {title}\n\
                                Text: {post}\n\
                                Summary 1: {summary0}\n\
                                Summary 2: {summary1}\n\n\
                                Preferred Summary='\
                                .format(post=dataset['post'][position], title=dataset['title'][position], summary0=dataset['summary0'][position], summary1=dataset['summary1'][position])
    # example = '»»»» Example »»»» Text - We were best friends over 4 years ... Summary 1 - Broke up with best friend, should I wish her a happy birthday... And what do you think of no contact? Summary 2 - should I wish my ex happy birthday, I broke no contact, I’m trying to be more patient, I’m too needy, and I don’t want her to think I’ll keep being that guy. Preferred Summary=1 »»»» Follow the instructions and the example(s) above »»»»'
    # ending = 'ONLY ANSWER WITH 1 OR 2 AND NO EXPLANATION!'

    # prompt = detailed_preable + '\n\n' + example + '\n\n' + sample_to_annotate + '\n\n' + ending
    # prompt = base_preamble + '\n\n' + example + '\n\n' + sample_to_annotate + '\n\n' + ending
    # prompt = base_preamble + '\n\n' + detailed_preable + '\n\n' + sample_to_annotate + '\n\n' + ending

    chat_content = [
        # System
        {
            'role': 'system',
            'content': base_preamble + '\n\n' + detailed_preable 
        },
        # Example 1
        {
            'role': 'user',
            'content': 'Which of the following summaries is better? Consider the coherence, accuracy, coverage, and overall quality of each summary. Return only "1" or "2" without an explanation! \n\n\
                        Title: {title}\n\
                        Text: {post}\n\
                        Summary 1: {summary0}\n\
                        Summary 2: {summary1}\n\
                        Preferred Summary='\
                        .format(post=dataset['post'][0], title=dataset['title'][0], summary0=dataset['summary0'][0], summary1=dataset['summary1'][0])                    
        },
        {
            'role': 'assistant',
            'content': '2'
        },
        # Example 2
        {
            'role': 'user',
            'content': 'Which of the following summaries is better? Consider the coherence, accuracy, coverage, and overall quality of each summary. Return only "1" or "2" without an explanation! \n\n\
                        Title: {title}\n\
                        Text: {post}\n\
                        Summary 1: {summary0}\n\
                        Summary 2: {summary1}\n\
                        Preferred Summary='\
                        .format(post=dataset['post'][2], title=dataset['title'][2], summary0=dataset['summary0'][2], summary1=dataset['summary1'][2])                    
        },
        {
            'role': 'assistant',
            'content': '1'
        },
        # Example 3
        {
            'role': 'user',
            'content': 'Which of the following summaries is better? Consider the coherence, accuracy, coverage, and overall quality of each summary. Return only "1" or "2" without an explanation! \n\n\
                        Title: {title}\n\
                        Text: {post}\n\
                        Summary 1: {summary0}\n\
                        Summary 2: {summary1}\n\
                        Preferred Summary='\
                        .format(post=dataset['post'][4], title=dataset['title'][4], summary0=dataset['summary0'][4], summary1=dataset['summary1'][4])                    
        },
        {
            'role': 'assistant',
            'content': '2'
        },
        # Example 4
        {
            'role': 'user',
            'content': 'Which of the following summaries is better? Consider the coherence, accuracy, coverage, and overall quality of each summary. Return only "1" or "2" without an explanation! \n\n\
                        Title: {title}\n\
                        Text: {post}\n\
                        Summary 1: {summary0}\n\
                        Summary 2: {summary1}\n\
                        Preferred Summary='\
                        .format(post=dataset['post'][6], title=dataset['title'][6], summary0=dataset['summary0'][6], summary1=dataset['summary1'][6])                    
        },
        {
            'role': 'assistant',
            'content': '1'
        },
        # Example 5
        {
            'role': 'user',
            'content': 'Which of the following summaries is better? Consider the coherence, accuracy, coverage, and overall quality of each summary. Return only "1" or "2" without an explanation! \n\n\
                        Title: {title}\n\
                        Text: {post}\n\
                        Summary 1: {summary0}\n\
                        Summary 2: {summary1}\n\
                        Preferred Summary='\
                        .format(post=dataset['post'][9], title=dataset['title'][9], summary0=dataset['summary0'][9], summary1=dataset['summary1'][9])                    
        },
        {
            'role': 'assistant',
            'content': '2'
        },

        # Final Prompt
        {
            'role': 'user',
            'content': instructions + sample_to_annotate
        }
    ]
    payload = {
    'model': model,
    'messages': chat_content,
    'stream': False
}
    headers = {'Content-Type': 'application/json'}
    response = requests.post(f'http://{ip_adress}:11434/api/chat', data=json.dumps(payload), headers=headers)
 
    return response
    # return response.json()['message']['content']

In [225]:
ai_feedback_test = pd.DataFrame(columns=['human', 'ai', 'ai_reversed'])
ai_feedback_test['human'] = (pref_data_labeled['preference'][10:110]+1).copy()
ai_feedback_test = ai_feedback_test.reset_index(drop=True)

for i in range(100):
    ai_feedback_test.loc[i, 'ai'] = get_annotation(pref_data_labeled, i).json()['message']['content']
    ai_feedback_test.loc[i, 'ai_reversed'] = get_annotation(pref_data_labeled, i, reversed_order=True).json()['message']['content']



In [239]:
# save to feather
ai_feedback_test.to_feather('../data/preference_data/ai_feedback_test.feather')

In [235]:
# convert datatypes
ai_feedback_test['ai'] = ai_feedback_test['ai'].astype(int)
ai_feedback_test['ai_reversed'] = ai_feedback_test['ai_reversed'].astype(int)

ai_feedback_test.dtypes

human          int64
ai             int32
ai_reversed    int32
dtype: object

In [237]:
# correct labels (only normal order)
ai_feedback_test[(ai_feedback_test['human'] == ai_feedback_test['ai'])].shape

(53, 3)

In [238]:
# correct labels (normal and reversed order)
ai_feedback_test[(ai_feedback_test['human'] == ai_feedback_test['ai']) & (ai_feedback_test['ai'] != ai_feedback_test['ai_reversed'])]


,human,ai,ai_reversed
16,1,1,2
22,2,2,1
40,2,2,1
48,2,2,1
52,1,1,2


In [241]:
ai_feedback_test[(ai_feedback_test['ai'] == 1) & (ai_feedback_test['ai_reversed'] == 1)]

,human,ai,ai_reversed
0,2,1,1
1,1,1,1
2,1,1,1
4,1,1,1
5,1,1,1
...,...,...,...
95,2,1,1
96,2,1,1
97,2,1,1
98,1,1,1


In [1]:
import pandas as pd
ai_feedback_1 = pd.read_feather('data/ai_feedback-llama2:70b-chat-2024-04-13.feather')

In [8]:
ai_feedback_1.loc[15575]

human                      1
ai                      None
ai_reversed             None
duration_ai             None
duration_ai_reversed    None
response_ai             None
response_ai_reversed    None
Name: 15575, dtype: object